In [ ]:
# Develop a stock screener
# Gap Up strategy screener

In [3]:
# libs
import os
import pandas as pd
import requests
from dotenv import load_dotenv #pip3 install python-dotenv

# load secrets
load_dotenv()

In [7]:


def get_sp500_tickers_from_wikipedia():
    # wikipedia page that lists S&P 500 companies
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    
    # read the html
    tables = pd.read_html(url)
    
    # easy, scrape the first table 0
    sp500_table = tables[0]
    
    # extract the ticker symbols to a list to use later
    tickers = sp500_table['Symbol'].tolist()
    
    return tickers

sp500_tickers = get_sp500_tickers_from_wikipedia()

# look at the ticker symbols
print(sp500_tickers)

# print the total -> note there are more than 500 (503)
print(len(sp500_tickers))

['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ATVI', 'ADM', 'ADBE', 'ADP', 'AAP', 'AES', 'AFL', 'A', 'APD', 'AKAM', 'ALK', 'ALB', 'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AMD', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'ABC', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'AON', 'APA', 'AAPL', 'AMAT', 'APTV', 'ACGL', 'ANET', 'AJG', 'AIZ', 'T', 'ATO', 'ADSK', 'AZO', 'AVB', 'AVY', 'AXON', 'BKR', 'BALL', 'BAC', 'BBWI', 'BAX', 'BDX', 'WRB', 'BRK.B', 'BBY', 'BIO', 'TECH', 'BIIB', 'BLK', 'BK', 'BA', 'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO', 'BF.B', 'BG', 'CHRW', 'CDNS', 'CZR', 'CPT', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CTLT', 'CAT', 'CBOE', 'CBRE', 'CDW', 'CE', 'CNC', 'CNP', 'CDAY', 'CF', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CMA', 'CAG', 'COP', 'ED', 'STZ', 'CEG', 'COO', 'CPRT', 'GLW', 'CTVA', 'CSGP', 'COST', 'CTRA', 'CCI', 'CSX', 'C

In [ ]:
def get_stock_data(symbol):
    iex_key = os.getenv("IEX_API_KEY") 
    
    # define and append endpoints below
    quote_url = f'https://cloud.iexapis.com/v1/stock/{symbol}/quote?token={iex_key}'
    company_url = f'https://cloud.iexapis.com/v1/stock/{symbol}/company?token={iex_key}'
    float_url = f'https://cloud.iexapis.com/v1/data-points/{symbol}/FLOAT?token={iex_key}'
    
    # execute api calls with error handling
    try:
        quote_data = requests.get(quote_url).json()
        company_data = requests.get(company_url).json()
        float_data = requests.get(float_url).json()
    except:
        return None  # Return None if there's an error fetching the data

    # data mapping
    stock_data = {
        'ticker': symbol,
        'stock_price': quote_data.get('latestPrice', None),
        'change_close': quote_data.get('change', None),  # KEY-Price Gap
        'change_close_pct': quote_data.get('changePercent', None),
        'volume_today': quote_data.get('latestVolume', None),
        'volume_avg_5d': None, 
        'market_cap': quote_data.get('marketCap', None),
        'float': float_data,  
        'float_pct': None, 
        'volatility': None,
        'liquidity': None,
        'news': None,
        'sector': company_data.get('sector', None),
        'industry': company_data.get('industry', None),
        'company_name': company_data.get('companyName', None),
        'latestUpdate': quote_data.get('latestUpdate', None)
    }

    return stock_data

# Fetch data for all S&P 500 tickers and store in a list
# For demonstration purposes, I'll just use a subset of tickers from the earlier defined list.
data_list = []
for ticker in sp500_tickers[:5]:  # Limiting to first 5 tickers for demonstration
    stock_data = get_stock_data(ticker)
    if stock_data:
        data_list.append(stock_data)

# Convert the list of dictionaries into a DataFrame
df_stocks = pd.DataFrame(data_list)

df_stocks
